# 최적 셔틀버스 노선 탐색

## 데이터 준비

In [54]:
import os
import pandas as pd

# 'data/raw_data' 경로 설정
root_dir = 'data/'
visitor_city = pd.read_csv(os.path.join(root_dir, "city_of_festival_visitors.csv"))  # 무주축제 방문객 top 18 지역들 (시군구명,od_cnts,시도명,행정동코드,위도,경도)
address = pd.read_csv(os.path.join(root_dir, "address_with_lon_lat_final.csv"))  # 행정동코드 + 위도경도 (행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도)
mooju = set(list(address[address['시군구명'] == '무주군']['행정동코드']))  # 무주군 행정동코드
other_city = list(address.merge(visitor_city, on=['시도명', '시군구명'])['행정동코드_x'])  # 다른 지역들 행정동코드 모음

In [55]:
visitor_city['시도 시군구'] = visitor_city['시도명'].fillna('') + ' ' + visitor_city['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
visitor_city['시도 시군구'] = visitor_city['시도 시군구'].str.strip()
visitor_city.head()

,시군구명,od_cnts,시도명,행정동코드,위도,경도,시도 시군구
0,전주시 덕진구,2834,전라북도,4511300000,35.847561,127.117672,전라북도 전주시 덕진구
1,서구,1813,대전광역시,3017000000,36.355179,127.383849,대전광역시 서구
2,전주시 완산구,1561,전라북도,4511100000,35.795512,127.132447,전라북도 전주시 완산구
3,유성구,1379,대전광역시,3020000000,36.362073,127.356410,대전광역시 유성구
4,동구,1207,대전광역시,3011000000,35.886500,128.635500,대전광역시 동구


In [56]:
address['시도 시군구'] = address['시도명'].fillna('') + ' ' + address['시군구명'].fillna('')

# 양쪽 값이 모두 null인 경우 빈 문자열 처리
address['시도 시군구'] = address['시도 시군구'].str.strip()
address

,행정동코드,시도명,시군구명,읍면동명,동리명,위도,경도,시도 시군구
0,1100000000,서울특별시,NaN,NaN,서울특별시,37.566679,126.978291,서울특별시
1,1111000000,서울특별시,종로구,NaN,종로구,37.580695,126.982799,서울특별시 종로구
2,1111051500,서울특별시,종로구,청운효자동,세종로,37.579997,126.976930,서울특별시 종로구
3,1111051500,서울특별시,종로구,청운효자동,옥인동,37.583480,126.963850,서울특별시 종로구
4,1111051500,서울특별시,종로구,청운효자동,누하동,37.578998,126.967561,서울특별시 종로구
...,...,...,...,...,...,...,...,...
21772,5183035000,강원특별자치도,양양군,강현면,정암리,38.143050,128.607330,강원특별자치도 양양군
21773,5183035000,강원특별자치도,양양군,강현면,용호리,38.132320,128.610700,강원특별자치도 양양군
21774,5183035000,강원특별자치도,양양군,강현면,전진리,38.124830,128.624220,강원특별자치도 양양군
21775,5183035000,강원특별자치도,양양군,강현면,물치리,38.158083,128.608889,강원특별자치도 양양군


In [57]:
from tqdm import tqdm

data_frames = []
muju_codes = [4573000000, 4573025000, 4573031000, 4573032000, 4573033000, 4573034000, 4573035000]

for i in tqdm(range(2, 11)):
    # print(f"{i}번째 ", end=" ")
    index = str(i).rjust(2, '0')
    df_tmp = pd.read_csv(f"EDA/jiwon/data/od_20230901_10/od_202309{index}_1.csv")
    to_muju = df_tmp['dest_hdong_cd'].isin(muju_codes) & (df_tmp['dest_purpose'] == 5)
    data_frames.append(df_tmp[to_muju])
del df_tmp

df_od = pd.concat(data_frames, axis=0)
df_od

  0%|          | 0/9 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:37<00:00,  4.20s/it]


,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
1096,4311374100,4573031000,20230902,09:00,11:00,1,0,0.0,0.0,5,163782,109,6
2269,4573033000,4573025000,20230902,20:00,20:00,0,3,1.0,5.0,5,20247,28,9
3874,4812965000,4573034000,20230902,16:00,18:00,0,4,0.0,4.0,5,424072,131,7
4218,4817073000,4573032000,20230902,13:00,16:00,1,0,0.0,0.0,5,119100,141,9
4773,4573032000,4573025000,20230902,17:00,19:00,0,4,0.0,5.0,5,128030,118,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237279,4574036000,4573034000,20230910,12:00,12:00,0,1,1.0,5.0,5,38926,27,5
3238635,4573033000,4573025000,20230910,12:00,12:00,1,5,0.0,5.0,5,9476,13,5
3239981,4572025000,4573033000,20230910,18:00,18:00,0,3,0.0,5.0,5,50623,33,6
3246027,4573025000,4573034000,20230910,11:00,12:00,0,3,0.0,5.0,5,46949,51,5


In [58]:
data_frames = []
for i in tqdm(range(2, 11)):
    # print(f"9월 {i}일 ", end=" ")
    index = str(i).rjust(2, '0')
    df_tmp = pd.read_csv(f"EDA/jiwon/data/stay_20230901_15/stay_202309{index}_1.csv")
    residents = df_tmp[(df_tmp['purpose'] == 0)]
    grouped_residents = residents.groupby(['hdong_cd', 'date', 'age'])['stay_cnts'].sum().reset_index()
    data_frames.append(grouped_residents)
del df_tmp

df_stay = pd.concat(data_frames, axis=0)
df_stay

100%|██████████| 9/9 [00:18<00:00,  2.08s/it]


,hdong_cd,date,age,stay_cnts
0,1111051500,20230902,0,15738
1,1111051500,20230902,1,26248
2,1111051500,20230902,2,21350
3,1111051500,20230902,3,32200
4,1111051500,20230902,4,37772
...,...,...,...,...
31252,5183035000,20230910,4,5202
31253,5183035000,20230910,5,7813
31254,5183035000,20230910,6,6296
31255,5183035000,20230910,7,5745


### 지역별 거주인원

In [59]:
# 지역별 전체 거주민 수
df_stay_all = df_stay.groupby(['hdong_cd', 'date'])['stay_cnts'].sum().reset_index()  # 날짜별 거주인구 합산
avg_stay_cnts_all = round(df_stay_all.groupby('hdong_cd')['stay_cnts'].mean().reset_index(), 0)  # 하루 평균 거주인구
avg_stay_cnts_all.head()

,hdong_cd,stay_cnts
0,1111051500,205476.0
1,1111053000,101129.0
2,1111054000,38539.0
3,1111055000,135173.0
4,1111056000,183499.0


In [60]:
# 지역별 20대,50대,60대 거주민 수
df_stay_age = df_stay[df_stay['age'].isin([2,5,6])]  # 날짜별 20대 거주인구
avg_stay_cnts_age = round(df_stay_age.groupby(['hdong_cd'])['stay_cnts'].mean().reset_index(), 0)  # 하루 평균 20대 거주인구
avg_stay_cnts_age.head()

,hdong_cd,stay_cnts
0,1111051500,22012.0
1,1111053000,11963.0
2,1111054000,5773.0
3,1111055000,18550.0
4,1111056000,24343.0


### 지역별 무주축제방문객 인원

In [61]:
df_od.head(3)

,origin_hdong_cd,dest_hdong_cd,date,start_time,end_time,gender,age,modal,origin_purpose,dest_purpose,od_dist_avg,od_duration_avg,od_cnts
1096,4311374100,4573031000,20230902,09:00,11:00,1,0,0.0,0.0,5,163782,109,6
2269,4573033000,4573025000,20230902,20:00,20:00,0,3,1.0,5.0,5,20247,28,9
3874,4812965000,4573034000,20230902,16:00,18:00,0,4,0.0,4.0,5,424072,131,7


In [62]:
# 그룹화
df_od_group = df_od.groupby(['origin_hdong_cd', 'date', 'age'])['od_cnts'].sum().reset_index()
df_od_group

,origin_hdong_cd,date,age,od_cnts
0,1117068500,20230908,0,12
1,1120067000,20230902,4,6
2,1121584000,20230902,0,9
3,1132051100,20230909,4,6
4,1135064000,20230902,0,64
...,...,...,...,...
1703,5113066000,20230902,0,12
1704,5113067500,20230907,3,6
1705,5113067500,20230909,3,6
1706,5113067500,20230909,4,12


In [63]:
# 지역별 전체 무주축제방문객 수
df_od_all = df_od_group.groupby(['origin_hdong_cd', 'date'])['od_cnts'].sum().reset_index()  # 날짜별 방문객수 합산
sum_od_cnts_all = round(df_od_all.groupby(['origin_hdong_cd'])['od_cnts'].sum().reset_index(), 0)  # 해당 지역에서 온 전체 방문객수
sum_od_cnts_all.head()

,origin_hdong_cd,od_cnts
0,1117068500,12
1,1120067000,6
2,1121584000,9
3,1132051100,6
4,1135064000,70


In [64]:
# 지역별 20대,50대,60대 무주축제방문객 수
df_od_age = df_od_group[df_od_group['age'].isin([2,5,6])]
sum_od_cnts_age = round(df_od_age.groupby('origin_hdong_cd')['od_cnts'].sum().reset_index(), 0)
sum_od_cnts_age

,origin_hdong_cd,od_cnts
0,1135064000,6
1,1138051000,5
2,1141062000,7
3,1141072000,14
4,1144060000,8
...,...,...
310,4888034000,13
311,4888035000,6
312,4888037000,13
313,4888040000,17


## 함수

### 노드 간 정보

In [65]:
# 시도시군구명으로 위경도 좌표를 반환하는 함수
def get_coordinates(lst):
    coordinates = []
    for name in lst:
        try:
            # '시도 시군구'로 '위도', '경도' 가져옴
            target = address[address['시도 시군구'] == name][['위도','경도']].iloc[0]
            x, y = target.iloc[0], target.iloc[1]
            coordinates.append((x,y))
        except IndexError:
            # 해당 '시도 시군구'에 대한 데이터가 없는 경우
            print(f"{name} 지역의 위도, 경도 정보가 없습니다.")
    return coordinates

In [66]:
import yaml
import requests

# api 사용을 위한 key값 가져오기 
with open("config.yaml", "r") as file:
    config = yaml.safe_load(file)
naver_api_id = config['naver api']['id']
naver_api_key = config['naver api']['key']
kakao_api_key = config['kakao api']['key']

# 네이버 api 요청 함수
def naver_request(start, goal):
    # 요청
    url = 'https://naveropenapi.apigw.ntruss.com/map-direction/v1/driving'
    params = {
        'goal': f'{goal[1]},{goal[0]}',
        'start': f'{start[1]},{start[0]}',
    }
    headers = {
        'x-ncp-apigw-api-key-id': naver_api_id,
        'x-ncp-apigw-api-key': naver_api_key
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    data = response.json()
    if data['code'] == 0:
        if data['route']['traoptimal']:
            summary = data['route']['traoptimal'][0]['summary']
            distance = summary['distance']
            duration = summary['duration']
            return distance, duration
        else:
            print("응답 에러: 'traoptimal' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패: ", data['message'])
        return -1, -1

# 카카오 api 요청 함수
def kakao_request(start, goal):
    # 요청
    url = 'https://apis-navi.kakaomobility.com/v1/directions'
    params = {
        'origin': f'{start[1]},{start[0]}',
        'destination': f'{goal[1]},{goal[0]}',
        'alternatives': True,
    }
    headers = {
        'Authorization': f'KakaoAK {kakao_api_key}'
    }
    response = requests.get(url, headers=headers, params=params)

    # 응답
    if response.status_code == 200:
        data = response.json()
        if data['routes']:
            summary = data['routes'][0]['summary']
            distance = summary.get('distance')
            duration = summary.get('duration')
            return distance, duration
        else:
            print("응답 에러: 'routes' 데이터가 없습니다.")
            return -1, -1
    else:
        print("요청 실패")
        return -1, -1

In [67]:
# 노드 간 이동시간, 이동거리 테이블 반환하는 함수
def get_path_info(coordinates, request_func):
    n = len(coordinates)
    distance_table = [[0] * n for _ in range(n)]
    duration_table = [[0] * n for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if j > i:
                start = coordinates[i]
                goal = coordinates[j]
                distance, duration = request_func(start, goal)
                distance_table[i][j], distance_table[j][i] = distance, distance
                duration_table[i][j], duration_table[j][i] = duration, duration

    return distance_table, duration_table

### 가중치

In [68]:
# 각 지역 거주인원
def get_residents_num(lst):
    residents_num = []
    residents_num_256 = []
    for name in lst:
        print("지역명:", name)
        codes = address[address['시도 시군구'] == name]['행정동코드'].unique().tolist()
        print("관련 행정동코드: ", codes)
        # 지역 내 행정동코드들의 거주인원 합산
        cnt_all = 0
        cnt_256 = 0
        for code in codes:
            tmp_for_all = avg_stay_cnts_all[avg_stay_cnts_all['hdong_cd']==code]
            if not tmp_for_all.empty:
                cnt_all += tmp_for_all['stay_cnts'].iloc[0]
            tmp_for_256 = avg_stay_cnts_age[avg_stay_cnts_age['hdong_cd']==code]
            if not tmp_for_256.empty:
                cnt_256 += tmp_for_256['stay_cnts'].iloc[0]
        residents_num.append(int(cnt_all))
        residents_num_256.append(int(cnt_256))
        print("총 거주인원:", cnt_all, ",  20/50/60대 거주인원:", cnt_256, "\n")
    return residents_num, residents_num_256

# 대전 지역으로 테스트
daejun = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']

daejun_residents_num, daejun_residents_num_256 = get_residents_num(daejun)

print(daejun_residents_num)
print(daejun_residents_num_256)

지역명: 세종특별자치시
관련 행정동코드:  [3600000000, 3611000000, 3611025000, 3611031000, 3611032000, 3611033000, 3611034000, 3611035000, 3611036000, 3611037000, 3611038000, 3611039000, 3611051000, 3611051500, 3611051800, 3611052000, 3611052300, 3611052500, 3611053000, 3611054000, 3611055000, 3611055500, 3611055600, 3611056000, 3611057000, 3611058000]
총 거주인원: 3048131.0 ,  20/50/60대 거주인원: 290518.0 

지역명: 대전광역시 유성구
관련 행정동코드:  [3020000000, 3020052000, 3020052600, 3020052700, 3020053000, 3020054000, 3020054600, 3020054700, 3020054800, 3020055000, 3020057000, 3020058000, 3020060000, 3020061000]
총 거주인원: 3820791.0 ,  20/50/60대 거주인원: 424653.0 

지역명: 대전광역시 서구
관련 행정동코드:  [3017000000, 3017051000, 3017052000, 3017053000, 3017053500, 3017054000, 3017055000, 3017055500, 3017056000, 3017057000, 3017057500, 3017058100, 3017058200, 3017058600, 3017058700, 3017058800, 3017059000, 3017059300, 3017059600, 3017059700, 3017060000, 3017063000, 3017064000, 3017065000, 3017066000]
총 거주인원: 5536840.0 ,  20/50/60대 거주인원: 674439.0 

In [69]:
# 각 지역 방문객
def get_visitors_num(lst):
    visitors_num = []
    visitors_num_256 = []
    for name in lst:
        print("지역명:", name)
        codes = address[address['시도 시군구'] == name]['행정동코드'].unique().tolist()
        print("관련 행정동코드: ", codes)
        # 지역 내 행정동코드들의 방문객 합산
        cnt_all = 0
        cnt_256 = 0
        for code in codes:
            tmp_for_all = sum_od_cnts_all[sum_od_cnts_all['origin_hdong_cd']==code]
            if not tmp_for_all.empty:
                cnt_all += tmp_for_all['od_cnts'].iloc[0]
            tmp_for_256 = sum_od_cnts_age[sum_od_cnts_age['origin_hdong_cd']==code]
            if not tmp_for_256.empty:
                cnt_256 += tmp_for_256['od_cnts'].iloc[0]
        visitors_num.append(int(cnt_all))
        visitors_num_256.append(int(cnt_256))
        print("총 방문객 수:", cnt_all, ",  20/50/60대 방문객 수:", cnt_256, "\n")
    return visitors_num, visitors_num_256

# 대전 지역으로 테스트
daejun_visitors_num, daejun_visitors_num_256 = get_visitors_num(daejun)

print(daejun_visitors_num)
print(daejun_visitors_num_256)

지역명: 세종특별자치시
관련 행정동코드:  [3600000000, 3611000000, 3611025000, 3611031000, 3611032000, 3611033000, 3611034000, 3611035000, 3611036000, 3611037000, 3611038000, 3611039000, 3611051000, 3611051500, 3611051800, 3611052000, 3611052300, 3611052500, 3611053000, 3611054000, 3611055000, 3611055500, 3611055600, 3611056000, 3611057000, 3611058000]
총 방문객 수: 758 ,  20/50/60대 방문객 수: 41 

지역명: 대전광역시 유성구
관련 행정동코드:  [3020000000, 3020052000, 3020052600, 3020052700, 3020053000, 3020054000, 3020054600, 3020054700, 3020054800, 3020055000, 3020057000, 3020058000, 3020060000, 3020061000]
총 방문객 수: 895 ,  20/50/60대 방문객 수: 189 

지역명: 대전광역시 서구
관련 행정동코드:  [3017000000, 3017051000, 3017052000, 3017053000, 3017053500, 3017054000, 3017055000, 3017055500, 3017056000, 3017057000, 3017057500, 3017058100, 3017058200, 3017058600, 3017058700, 3017058800, 3017059000, 3017059300, 3017059600, 3017059700, 3017060000, 3017063000, 3017064000, 3017065000, 3017066000]
총 방문객 수: 1182 ,  20/50/60대 방문객 수: 304 

지역명: 대전광역시 대덕구
관련 행정동코드: 

In [70]:
# 전체 데이터에서 20대 어디서 오는지 확인
age20 = df_od[df_od['age']==2][['origin_hdong_cd', 'date', 'age', 'od_cnts']]
age20 = age20.groupby(['origin_hdong_cd'])['od_cnts'].sum().reset_index()
unique_address = address.drop_duplicates(subset=['행정동코드'])
age20eng = pd.merge(age20, unique_address[['행정동코드','시도 시군구']], 'left', left_on='origin_hdong_cd', right_on='행정동코드')
age20eng = age20eng[['origin_hdong_cd', '시도 시군구', 'od_cnts']]
age20eng

,origin_hdong_cd,시도 시군구,od_cnts
0,1135064000,서울특별시 노원구,6
1,1138051000,서울특별시 은평구,5
2,1141062000,서울특별시 서대문구,7
3,1141072000,서울특별시 서대문구,14
4,1144060000,서울특별시 마포구,8
...,...,...,...
273,4888025000,경상남도 거창군,45
274,4888034000,경상남도 거창군,8
275,4888037000,경상남도 거창군,8
276,4888040000,경상남도 거창군,7


In [71]:
# 지역별 20/50/60대 방문율
def get_proportion(a, b, n):
    lst = []
    for i in range(n):
        lst.append(a[i] / b[i])
    return lst

In [72]:
import numpy as np

def softmax(weights):
    e = np.exp(weights - np.max(weights))  # 오버플로 방지를 위해 최대값을 빼줌
    return e / e.sum()

def get_weights(nodes, weights):    
    residents_num, residents_num_256 = get_residents_num(nodes)
    visitors_num, visitors_num_256 = get_visitors_num(nodes)
    # 방식1: 해당 지역의 2/5/60대 거주민 중 축제방문객 비율. (2/5/60대 축제 관심도가 높은 곳을 우선)
    # weight_age = get_proportion(visitors_num_256, residents_num_256, len(nodes))
    # 방식2: 해당 지역의 전체 축제방문객 중 2/5/60대 비율. ()
    weight_age = get_proportion(visitors_num_256, visitors_num, len(nodes))
    # 방식3: 해당 지역의 거주민 중 2/5/60대 거주민 비율. (단순히 2/5/60대가 많은 곳을 우선)
    # weight_age = get_proportion(residents_num_256, residents_num, len(nodes))

    for i, w in enumerate(weight_age):
        weights[i] += w

    softmax_weights = softmax(weights)
    
    stations = {nodes: weight for nodes, weight in zip(nodes, softmax_weights)}
    print(stations)

    return stations

### 탐색

In [73]:
import networkx as nx

# 모든 경로 찾기
def find_all_routes(G, start, target, visited=None, path=None):
    if visited is None:
        visited = set()
    if path is None:
        path = []

    visited.add(start)
    path.append(start)

    if start == target:
        yield path.copy()
    else:
        for neighbor in G.neighbors(start):
            if neighbor not in visited:
                yield from find_all_routes(G, neighbor, target, visited, path)

    path.pop()
    visited.remove(start)

# 최적 경로 탐색 함수
def find_optimal_routes(G, stations, min_threshold, max_threshold, step):
    optimal_routes = []
    max_stops = 0
    explored_routes = set()  # 탐색된 경로를 저장할 집합

    # 여러 배수 값을 탐색
    for threshold in np.arange(min_threshold, max_threshold + step, step):

        for station in stations.keys():
            if station != "전라북도 무주군":  # 목적지가 아니면
                routes = list(find_all_routes(G, station, "전라북도 무주군"))

                for route in routes:
                    valid_route = True
                    total_distance = 0
                    total_time = 0
                    total_weight = 0  # 가중치 누적

                    # 각 경로의 모든 구간에 대해 확인
                    for i in range(len(route) - 1):
                        current_station = route[i]

                        # 현재 노드에서 무주군으로 바로 이동할 때의 거리와 시간
                        if "전라북도 무주군" in G[current_station]:  # 직접 경로가 있는 경우에만
                            direct_distance = G[current_station]["전라북도 무주군"]['distance']
                            direct_time = G[current_station]["전라북도 무주군"]['time']

                            # 현재 구간의 거리와 시간을 누적
                            total_distance += G[route[i]][route[i + 1]]['distance']
                            total_time += G[route[i]][route[i + 1]]['time']

                            # 각 중간 노드에서도 주어진 배수(factor) 조건 확인
                            if total_distance > direct_distance * threshold or total_time > direct_time * threshold:
                                valid_route = False
                                break

                        # 가중치 누적 (출발지 제외)
                        if i > 0:
                            total_weight += stations[current_station]

                    if valid_route:
                        # 경로를 집합에 저장할 수 있는 형태로 변환 (정렬하여 순서 상관없이 비교 하기 위함)
                        route_tuple = tuple(sorted(route))

                        # 만약 해당 경로가 이전 배수에서 이미 탐색되었다면 제외
                        if route_tuple not in explored_routes:
                            explored_routes.add(route_tuple)  # 새로운 경로로 저장

                            # 최대 정거장 수 확인
                            if len(route) > max_stops:
                                max_stops = len(route)
                                optimal_routes = [(route, total_distance, total_time, total_weight, threshold)]
                            elif len(route) == max_stops:
                                optimal_routes.append((route, total_distance, total_time, total_weight, threshold))

    return optimal_routes


In [74]:
def print_optimal_routes(optimal_routes, name):
    print(f"{name}지역 최적의 셔틀 노선")
    if optimal_routes:
        for route, total_distance, total_time, total_weight, threshold in optimal_routes:
            # 총 거리를 km로 변환하고 소수점 아래 두째 자리까지 포맷
            total_distance_km = total_distance / 1000  # 미터를 킬로미터로 변환
            total_distance_formatted = f"{total_distance_km:.2f} km"

            # 총 시간을 시간과 분으로 변환 (반올림)
            total_time_hours = total_time // 60
            total_time_minutes = round(total_time % 60)

            print(f"최적의 셔틀 노선: {route} | 총 거리: {total_distance_formatted} | 총 시간: {total_time_hours}시간 {total_time_minutes}분 | 총 가중치: {total_weight} | threshold: {threshold:.2f}")
    else:
        print("유효한 경로가 없습니다.")

---

## 메인

In [75]:
# 경유지 리스트
daejun = ['세종특별자치시', '대전광역시 유성구', '대전광역시 서구', '대전광역시 대덕구', '대전광역시 중구', '충청남도 금산군', '충청북도 영동군', '전라북도 무주군']
jeonbuk = ['전라북도 군산시', '전라북도 익산시', '전라북도 전주시 완산구', '전라북도 전주시 덕진구', '전라북도 진안군', '전라북도 장수군', '전라북도 무주군']
# 방문객 수에 근거한 노드별 초기 가중치
daejun_weights = [1, 2, 2, 1, 1, 1, 2, 0]  
jeonbuk_weights = [1, 1, 2, 2, 1, 2, 0]

In [76]:
def search(nodes, weights, name):
    # 1. 위경도 좌표 기반으로 경유지 간 이동거리, 이동시간 테이블 만듦

    # 경유지 위경도 좌표 
    coordinates = get_coordinates(nodes) 

    # 네이버 api
    # n_distance_table, n_duration_table = get_path_info(coordinates, naver_request)
    # n_duration_min_table = [[round(value / 3600, 2) for value in row] for row in n_duration_table]

    # 카카오 api
    k_distance_table, k_duration_table = get_path_info(coordinates, kakao_request)
    k_duration_min_table = [[round(value / 60, 2) for value in row] for row in k_duration_table]


    distance_table = k_distance_table
    duration_table = k_duration_min_table

    # 2. 정거장 후보지와 가중치
    stations = get_weights(nodes, weights)  

    # 3. 탐색
    # 그래프 생성
    G = nx.DiGraph()

    # 그래프에 엣지 추가
    for i, from_station in enumerate(stations.keys()):
        for j, to_station in enumerate(stations.keys()):
            if i != j:
                G.add_edge(from_station, to_station, 
                        distance=distance_table[i][j], 
                        time=duration_table[i][j])
                
    # 탐색 범위 설정 (예: 1.1부터 1.5까지 0.1 간격으로 탐색)
    min_factor = 1.1
    max_factor = 1.5
    step = 0.1

    # 최적 경로 탐색 실행
    optimal_routes = find_optimal_routes(G, stations, min_factor, max_factor, step)

    # 결과 출력
    print_optimal_routes(optimal_routes, name)

In [77]:
search(daejun, daejun_weights, "대전")

지역명: 세종특별자치시
관련 행정동코드:  [3600000000, 3611000000, 3611025000, 3611031000, 3611032000, 3611033000, 3611034000, 3611035000, 3611036000, 3611037000, 3611038000, 3611039000, 3611051000, 3611051500, 3611051800, 3611052000, 3611052300, 3611052500, 3611053000, 3611054000, 3611055000, 3611055500, 3611055600, 3611056000, 3611057000, 3611058000]
총 거주인원: 3048131.0 ,  20/50/60대 거주인원: 290518.0 

지역명: 대전광역시 유성구
관련 행정동코드:  [3020000000, 3020052000, 3020052600, 3020052700, 3020053000, 3020054000, 3020054600, 3020054700, 3020054800, 3020055000, 3020057000, 3020058000, 3020060000, 3020061000]
총 거주인원: 3820791.0 ,  20/50/60대 거주인원: 424653.0 

지역명: 대전광역시 서구
관련 행정동코드:  [3017000000, 3017051000, 3017052000, 3017053000, 3017053500, 3017054000, 3017055000, 3017055500, 3017056000, 3017057000, 3017057500, 3017058100, 3017058200, 3017058600, 3017058700, 3017058800, 3017059000, 3017059300, 3017059600, 3017059700, 3017060000, 3017063000, 3017064000, 3017065000, 3017066000]
총 거주인원: 5536840.0 ,  20/50/60대 거주인원: 674439.0 

In [78]:
search(jeonbuk, jeonbuk_weights, "전북")

지역명: 전라북도 군산시
관련 행정동코드:  [4513000000, 4513025000, 4513031000, 4513032000, 4513033000, 4513034000, 4513035000, 4513036000, 4513037000, 4513038000, 4513039000, 4513040000, 4513051500, 4513053000, 4513055000, 4513056000, 4513060500, 4513064000, 4513065000, 4513066000, 4513067000, 4513068000, 4513069000, 4513070100, 4513070200, 4513070300, 4513071000, 4513072000]
총 거주인원: 3132379.0 ,  20/50/60대 거주인원: 378252.0 

지역명: 전라북도 익산시
관련 행정동코드:  [4514000000, 4514025000, 4514031000, 4514032000, 4514033000, 4514034000, 4514035000, 4514036000, 4514037000, 4514038000, 4514039000, 4514040000, 4514041000, 4514042000, 4514043000, 4514044000, 4514052000, 4514053000, 4514056000, 4514057000, 4514058000, 4514059500, 4514061000, 4514062000, 4514064600, 4514064700, 4514065200, 4514065600, 4514067000, 4514069000]
총 거주인원: 3328022.0 ,  20/50/60대 거주인원: 420440.0 

지역명: 전라북도 전주시 완산구
관련 행정동코드:  [4511100000, 4511151000, 4511153000, 4511160500, 4511163500, 4511165000, 4511166000, 4511167100, 4511167200, 4511168000, 451116